<a href="https://colab.research.google.com/github/thedatasense/llm-healthcare/blob/main/MIMIC_GPT_Evaluation_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
!pip install -q  sqlalchemy cockroachdb pandas psycopg2-binary matplotlib

In [2]:
!curl --create-dirs -o $HOME/.postgresql/root.crt 'https://cockroachlabs.cloud/clusters/5bbbe91d-b65e-410e-a783-597c93f501f6/cert'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2728    0  2728    0     0   8404      0 --:--:-- --:--:-- --:--:--  8419


In [3]:
import pandas as pd
from IPython.display import clear_output
from sqlalchemy.engine import create_engine
#from datasets import load_dataset
from openai import OpenAI
import io
import base64
import random
import requests
import torch
from PIL import Image
#from transformers import AutoProcessor,Qwen2_5_VLForConditionalGeneration
#from qwen_vl_utils import process_vision_info
import os
import pandas as pd
from sqlalchemy.engine import create_engine
#from transformers import AutoProcessor, BitsAndBytesConfig
import json
import yaml
import platform
from sqlalchemy import text
from IPython.display import display,clear_output
import time
import json


In [4]:
cnfig_file="/Users/bineshkumar/Documents/config.yaml"
def get_from_cnfg(key_path,file_path=cnfig_file):
    try:
        with open(file_path, 'r') as file:
            data = yaml.safe_load(file)

        keys = key_path.split('.')
        value = data
        for key in keys:
            value = value[key]
        return value

    except FileNotFoundError:
        print(f"File {file_path} not found")
    except yaml.YAMLError as e:
        print(f"YAML parsing error: {e}")
    except KeyError:
        print(f"Key path {key_path} not found")
    except Exception as e:
        print(f"Error: {e}")
    return None
os_name = platform.system()

In [5]:
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    from google.colab import userdata
    engine = create_engine(userdata.get('DB_URL'))
    gem_key=userdata.get('DB_URL')
    oai_key=userdata.get('DB_URL')
    source_folder='/content/drive/MyDrive/Health_Data/MIMIC_JPG_AVL/mimic-cxr-jpg/2.1.0/files/'
elif os_name == "Darwin":
    cnfig_file="/Users/bineshkumar/Documents/config.yaml"
    DB_URL = get_from_cnfg("cd_url",cnfig_file)
    gem_key=get_from_cnfg("gem_token",cnfig_file)
    oai_key=get_from_cnfg("oai_token",cnfig_file)
    source_folder='/Users/bineshkumar/Documents/mimic-cxr-jpg/2.1.0/files/'
elif os_name == "Linux":
    DB_URL = get_from_cnfg("cd_url",cnfig_file)
    gem_key=get_from_cnfg("gem_token",cnfig_file)
    oai_key=get_from_cnfg("oai_token",cnfig_file)
    source_folder=""
engine = create_engine(DB_URL)

In [6]:
def insert_model_response(engine, uid,question_id,question, question_category, actual_answer, model_name, model_answer, image_link):
    from sqlalchemy import text
    with engine.connect() as conn:
        trans = conn.begin()
        try:
            conn.execute(text("""
                INSERT INTO model_responses_r2
                (uid,question_id,question, question_category, actual_answer, model_name, model_answer, image_link)
                VALUES (:uid,:question_id,:question, :question_category, :actual_answer, :model_name, :model_answer, :image_link)
            """), {
                "uid": uid,
                "question_id": question_id,
                "question": question,
                "question_category": question_category,
                "actual_answer": actual_answer,
                "model_name": model_name,
                "model_answer": model_answer,
                "image_link": image_link
            })
            trans.commit()  # Commit the transaction
        except Exception as e:
            trans.rollback()
            raise e

In [7]:
def check_duplicate(engine,uid,question_id,question, question_category, model_name,image_link):
    query = text("""
        SELECT 1 FROM model_responses_r2
        WHERE
        uid = :uid
        AND question_id = :question_id and
        question = :question
          AND question_category = :question_category
          AND model_name = :model_name
          AND image_link = :image_link
        LIMIT 1
    """)
    with engine.connect() as conn:
        result = conn.execute(query, {
            "uid": uid,
            "question_id": question_id,
            "question": question,
            "question_category": question_category,
            "model_name": model_name,
            "image_link": image_link
        }).fetchone()
    return result is not None

In [8]:
def fetch_generation_data(engine):
    import pandas as pd
    import re
    from sqlalchemy import text
    from sqlalchemy.dialects.postgresql.base import PGDialect
    def fake_get_server_version_info(self, connection):
        version_str = connection.execute(text("SELECT version()")).scalar()
        match = re.search(r'v(\d+)\.(\d+)\.(\d+)', version_str)
        if match:
            return tuple(map(int, match.groups()))
        return (13, 0, 0)
    PGDialect._get_server_version_info = fake_get_server_version_info
    query = f"SELECT id,question_id,condition as question_type, text as question,answer as ground_truth,image from mimic_all_qns; "
    return pd.read_sql(query, con=engine)

In [9]:
def encode_image_stream(image_path):
    if os.path.exists(image_path):
        with open(image_path, "rb") as img_file:
            return base64.b64encode(img_file.read()).decode("utf-8")
    else:
        return None

def generate_gpt_response(prompt_text, image_link):
    base64_image = encode_image_stream(image_link)
    
    if base64_image is None:
        return None
        
    client = OpenAI(api_key=oai_key)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an expert medical professional. "
                    "When responding, provide a concise explanation of the image findings. "
                    "For example, if asked about abnormalities, answer briefly with terms like 'atelectasis, lung opacity'."
                )
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt_text,
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                    },
                ],
            },
        ],
    )
    return response.choices[0].message.content

def save_failed_images(failed_list, filename="failed_images_gpt4o.txt"):
    with open(filename, "w") as f:
        for img in failed_list:
            f.write(f"{img}\n")

In [10]:
import random
from IPython.display import Image, display

def print_a_sample(df,prompt_prefix,source_folder=source_folder,width=250):
    # Get a random row as a Series
    row = df.sample(n=1).iloc[0]  # Add .iloc[0] to get the row as a Series

    # Now these will return scalar values
    question = row["question"]
    actual_answer = row["ground_truth"]
    image_link = source_folder + row["image"]

    print(f"{question}")
    print(f"GT: {actual_answer}")
    print(f"Image: {image_link}")


    # Uncommented to display the image
    display(Image(filename=image_link, width=width))
    generated_answer = generate_gpt_response(prompt_prefix + " " + row["question_type"] + ":" + row["question"] + prompt_prefix, image_link)
    print(f"gpt-4o : {generated_answer}")
    print('--------------------------------')


In [11]:
from IPython.display import clear_output
model_id = "gpt-4o"
failed_images = []

for index, row in fetch_generation_data(engine).iterrows():
    uid = row["id"]
    question_id = row["question_id"]
    question_category = row["question_type"]
    question = row["question"]
    actual_answer = row["ground_truth"]
    image_link = source_folder + row["image"]
    
    if check_duplicate(engine, uid, str(question_id), question, question_category, model_id, image_link):
        print(f"Duplicate record found for question: {question}. Skipping generation.")
        clear_output(wait=True)
        continue
    
    try:
        generated_answer = generate_gpt_response(question, image_link)
        
        if generated_answer is None:
            print(f"Image not found: {image_link}")
            failed_images.append(image_link)
            save_failed_images(failed_images)
            continue
            
        print(f"{model_id} : {generated_answer}")
        print(f"GT: {actual_answer}")
        insert_model_response(engine, uid, question_id, question, question_category, actual_answer, model_id, generated_answer, image_link)
    except Exception as e:
        print(f"Error generating response for question: {question}. Error: {e}")
        continue
    
    time.sleep(5)
    print('--------------------------------')
    clear_output(wait=True)

save_failed_images(failed_images)

Error generating response for question: What prosthetic features might be identified on the chest X-ray image?. Error: (psycopg2.OperationalError) could not receive data from server: Operation timed out
SSL SYSCALL error: Operation timed out

[SQL: 
                INSERT INTO model_responses_r2
                (uid,question_id,question, question_category, actual_answer, model_name, model_answer, image_link)
                VALUES (%(uid)s,%(question_id)s,%(question)s, %(question_category)s, %(actual_answer)s, %(model_name)s, %(model_answer)s, %(image_link)s)
            ]
[parameters: {'uid': 'ffd60688-5da7c1d3-4229e284-c84ba788-c00f4302', 'question_id': 9653, 'question': 'What prosthetic features might be identified on the chest X-ray image?', 'question_category': 'original', 'actual_answer': 'The image may show structures consistent with aortic and tricuspid valve replacements, which could appear as circular or semi-circular opacities in the region of the heart valves.', 'model_name

OperationalError: (psycopg2.OperationalError) could not translate host name "uneven-dingo-4829.jxf.gcp-us-east1.cockroachlabs.cloud" to address: nodename nor servname provided, or not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)